In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

import glob
import os
import sys

from tqdm.notebook import tqdm
from datetime import datetime

import Cross_Eval

In [3]:
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import FunctionTransformer, RobustScaler, MinMaxScaler, StandardScaler, LabelBinarizer
from sklearn.preprocessing import normalize, robust_scale, minmax_scale

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier

from sklearn.kernel_approximation import RBFSampler, Nystroem

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate, GroupKFold, KFold

from sklearn.metrics import make_scorer, confusion_matrix, roc_auc_score, roc_curve, plot_confusion_matrix, f1_score, recall_score, accuracy_score

from sklearn.multiclass import OneVsRestClassifier

from mlxtend.evaluate import scoring

# Preparing Metadata/Labels

In [4]:
all_files = pd.DataFrame(glob.glob("/mnt/b/Git_Projects/Biospec_Analysis/Extracted_Spectra/*.pickle"), columns = ["hdf_loc"])
all_files["TMA"] = all_files.apply(lambda row: float(row["hdf_loc"].split("/")[-1][4:5]), axis = 1)
all_files["Core"] = all_files.apply(lambda row: row["hdf_loc"].split("/")[-1][8:-7], axis = 1)

In [5]:
metadata = pd.read_excel("/mnt/b/Google_Drive/PhD/Project with Janet/Scoring grids Advancing Front (TMAs 1-3).xlsx"
              , sheet_name = "reformatted")

metadata = metadata.merge(all_files, left_on = ["TMA", "TMA site"], right_on = ["TMA", "Core"])
print(metadata.shape)

(79, 18)


In [6]:
new_metadata = pd.read_excel("/mnt/b/Google_Drive/PhD/Project with Janet/new_metadata.xlsx")

In [7]:
new_metadata["DODvsAlive"] = new_metadata["OUTCOME"].map(
                            {"DOD" : "DOD",
                             "alive": "Other",
                             "died other" : "Other",
                             "died" : "Other",
                             "Died" : "Other",
                             "Died other": "Other"})

new_metadata["DiedvsAlive"] = new_metadata["OUTCOME"].map(
                            {"DOD" : "Died",
                             "alive": "Alive",
                             "died other" : "Died",
                             "died" : "Died",
                             "Died" : "Died",
                             "Died other": "Died"})

new_metadata["ECS"] = new_metadata["ECS"].map(
                            {"y" : "Y",
                             "Y": "Y",
                             "N" : "N",
                             "n" : "N"})

In [8]:
new_metadata['lifespan'] = new_metadata['Date of death'] - new_metadata['date of diag']

new_metadata["2year"] = new_metadata['lifespan'].dt.days < 730
new_metadata["5year"] = new_metadata['lifespan'].dt.days < 1825

In [9]:
new_metadata["2year"] = new_metadata.apply(lambda row: True if (row['Date of death']-row['date of diag']).days<730 and row.OUTCOME!='Alive' else False, axis = 1)
new_metadata["5year"] = new_metadata.apply(lambda row: True if 730<(row['Date of death']-row['date of diag']).days<1825 and row.OUTCOME!='Alive' else False, axis = 1)

new_metadata["Alive"] = new_metadata.apply(lambda row: True if 730<(row['Last seen/reviewed']-row['date of diag']).days<1825 and row.OUTCOME=='Alive' else False, axis = 1)

In [10]:
new_metadata.head()

,Patient_Number,gender,date of diag,age @ diag,site,pT,pN,stage,ECS,OUTCOME,...,site of recurrence,Last seen/reviewed,Last updated,OUTCOME.updated by,DODvsAlive,DiedvsAlive,lifespan,2year,5year,Alive
0,3143,m,2004-03-29,58,floor of mouth,4,0,4,N,DOD,...,local,NaT,2020-05-11,FGD,DOD,Died,3341 days,False,False,False
1,3167,m,2004-09-30,74,floor of mouth,2,2,4,Y,DOD,...,regional only,NaT,2012-10-04,CBarry,DOD,Died,390 days,True,False,False
2,3169,m,2004-10-28,63,floor of mouth,3,1,3,N,died other,...,NaN,NaT,2006-12-01,RJS,Other,Died,652 days,True,False,False
3,3170,m,2004-11-22,59,other,4,2b,4a,Y,Died other,...,NaN,NaT,2020-04-01,FGD,Other,Died,3690 days,False,False,False
4,3230,m,2006-01-26,74,tongue,2,1,3,Y,DOD,...,local and regional,NaT,2011-05-01,AS,DOD,Died,161 days,True,False,False


In [11]:
new_metadata.site.value_counts()

tongue            38
floor of mouth    34
other             28
oropharynx         7
Name: site, dtype: int64

Import ASMA data

In [12]:
asma = pd.read_excel("/mnt/b/Google_Drive/PhD/Project with Janet/ASMA.xlsx", sheet_name = "Sheet3")
new_metadata = new_metadata.merge(asma, left_on = "Patient_Number", right_on = "Case ID")

In [13]:
from Preprocessing_Methods import *

import sys

sys.path.append("/mnt/b/Git_Projects/Biospec_Analysis/UNET")

In [14]:
select = ['Tumour']

total = pd.concat({"{}-{}".format(row["TMA"], row["Core"]):
                   truncate(pd.read_pickle(row["hdf_loc"]).query(f"Tissue in {select}").sample(frac = 1), start = 1000, end = 1800)
                   for name, row in tqdm(list(metadata.iterrows()))}, names = ["ID"])

wn_cols = total.columns
#total = total.merge(new_metadata, left_on = "Patient nu ", right_on = "Patient Number")
#total = total.drop(np.nan, level = "Overall Death ")

In [15]:
total = total.reset_index().merge(new_metadata, left_on = "Patient nu ", right_on = "Patient_Number", how = 'inner')
total = total.set_index(list(total.columns.difference(wn_cols))).dropna(axis = 1)

In [16]:
total

998.0   \
2year 5year ASMA Alive Annotation_loc                                     Case ID Core_x Core_y Corrected Site AJCC/UICC DODvsAlive Date of death Date of recurrence Diagnosis Age DiedvsAlive ECS_x ECS_y ID      Last seen/reviewed Last updated N stage OUTCOME OUTCOME.updated by Overall Death  Pathogical Stage Patient nu  Patient_Number Recurrence? T stage TMA TMA site Tissue advancing front type age @ diag date of diag diff envi_loc                                           gender_x gender_y lifespan pN pT radiotherapy recurrence site       site of recurrence stage survival (months)             
True  False H    False /mnt/b/Google_Drive/PhD/Project with Janet/Labe... 3540    G13    G13    other                    DOD        2010-03-20    2010-03-02         85.0          Died        yes   Y     1.0-G13 NaT                2020-04-21   2b      DOD     PG                 yes            pT4N2bM0         3540        3540           yes         4       1.0 G13      Tumour non-cohesive         85         2009-10-19   mod  /mnt/b/Google_Drive/PhD/Project with Janet/Kohl... m        m        152 days 2b 4  NaN          yes        other      regional           4a    5.0                0.030717   
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           5.0                0.019120   
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           5.0                0.019243   
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           5.0                0.028798   
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           5.0                0.033685   
...                                                                                                                                                                                                                                                                                                                                                                                       